## Reserve and configure resources on KVM

## Create lease and reserve resources

In [1]:
import json
import os
import chi
import chi.ssh
import chi.network
from datetime import datetime, timedelta
from dateutil import tz
import chi.server

In [2]:
import chi,os

site_name = "KVM@TACC"
chi.use_site(site_name)
os.environ["OS_REGION_NAME"] = site_name
project = "CHI-231095"
#os.getenv("OS_PROJECT_NAME")
print(project)
chi.set('project_name', project)

Now using KVM@TACC:
URL: https://kvm.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org
CHI-231095


In [3]:
username = os.getenv("USER")
suffix = username + "_k8s-ml"
server_prefix = "node" 
network_name = "net-" + suffix
subnet_name = "subnet-" + suffix
router_name = "router-" + suffix
lease_name = "lease-" + suffix

In [4]:
#Server attributes
image_name='CC-Ubuntu20.04'
node_type="compute_cascadelake_r"
server_count=3
physical_network="physnet1"
subnet_cidr = '192.168.1.0/24'

#### Create the Network

In [5]:
os_conn = chi.clients.connection()

In [6]:
network = os_conn.create_network(
                network_name,
            )
network_id = network['id']

print(f'Network ID: {network_id}')


Network ID: 8a4089dd-5f31-49b9-aaec-25d7e87bc398


#### Configure the Network

In [7]:
subnet = chi.network.create_subnet(subnet_name, network_id, cidr=subnet_cidr, gateway_ip=None)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router(router['id'], subnet['id'])

{'network_id': '8a4089dd-5f31-49b9-aaec-25d7e87bc398',
 'tenant_id': '287158fd03da48e48eb0d2682f38fcc9',
 'subnet_id': 'a5a89372-e36f-4759-8fd1-0373033907d1',
 'subnet_ids': ['a5a89372-e36f-4759-8fd1-0373033907d1'],
 'port_id': 'b7271a63-a2b3-4882-ba4d-948c90c7c086',
 'id': '696dce19-c36f-4d84-be36-4fbdb1ee205d'}

#### Start the server

In [8]:
servers = []
for i in range(server_count):
    servers.append(chi.server.create_server(server_prefix+f"-{i}-" + suffix, 
                                  network_id=network_id,
                                  nics=[{"net-id": network_id,"v4-fixed-ip": f"192.168.1.2{i}" }],
                                  image_name=image_name,
                                            flavor_name= "m1.large",
                                            
                                  count=1))

#### Associate floating ips

In [9]:
nodes = {}

for j,i in enumerate(servers):
    chi.server.wait_for_active(i.id)
    floating_ip = chi.server.associate_floating_ip(i.id)
    nodes[j] = floating_ip

    print(f'Floating IP of node_{j}: {floating_ip}')

Floating IP of node_0: 129.114.26.122
Floating IP of node_1: 129.114.27.104
Floating IP of node_2: 129.114.25.34


#### Create and add security groups to all servers

In [ ]:
%%bash
export OS_AUTH_URL=https://kvm.tacc.chameleoncloud.org:5000/v3

access_token=$(curl -s -H"authorization: token $JUPYTERHUB_API_TOKEN"     "$JUPYTERHUB_API_URL/users/$JUPYTERHUB_USER"     | jq -r .auth_state.access_token)
export OS_ACCESS_TOKEN="$access_token"
SECURITY_GROUP_NAME="Allow SSH"

if ! openstack security group show "$SECURITY_GROUP_NAME" > /dev/null 2>&1; then
    echo "Security group does not exist yet - creating it for you now"
    openstack security group create "$SECURITY_GROUP_NAME"  --description "Enable SSH traffic on TCP port 22"
    openstack security group rule create "$SECURITY_GROUP_NAME" \
     --protocol tcp --dst-port 22:22 --remote-ip 0.0.0.0/0

else
    echo "Security group already exists"
fi

In [ ]:
%%bash
export OS_AUTH_URL=https://kvm.tacc.chameleoncloud.org:5000/v3

access_token=$(curl -s -H"authorization: token $JUPYTERHUB_API_TOKEN"     "$JUPYTERHUB_API_URL/users/$JUPYTERHUB_USER"     | jq -r .auth_state.access_token)
export OS_ACCESS_TOKEN="$access_token"
SECURITY_GROUP_NAME="Allow http"

if ! openstack security group show "$SECURITY_GROUP_NAME" > /dev/null 2>&1; then
    echo "Security group does not exist yet - creating it for you now"
    openstack security group create "$SECURITY_GROUP_NAME"  --description "Enable http traffic on TCP port 32000"
    openstack security group rule create "$SECURITY_GROUP_NAME" \
     --protocol tcp --dst-port 5000:5000 --remote-ip 0.0.0.0/0

else
    echo "Security group already exists"
fi

In [10]:
for j,i in enumerate(servers):
    nova_server = chi.nova().servers.get(i.id)
    nova_server.add_security_group("Allow SSH")
    nova_server.add_security_group("Allow http")
    print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['Allow SSH', 'Allow http', 'default']
updated security groups: ['Allow SSH', 'Allow http', 'default']
updated security groups: ['Allow SSH', 'Allow http', 'default']


#### Wait for SSH access to all servers

In [11]:
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)

### Configuring the servers

#### Add your public keys to each of the server

In [12]:
from chi.ssh import Remote
for node_ip in nodes.values():
    remote = Remote(node_ip)
    nova=chi.clients.nova()
    [kp.name for kp in nova.keypairs.list()]
    keypair_name = 'id_rsa_chameleon' # The name of the key they uploaded 
    public_key = nova.keypairs.get(keypair_name).public_key # Contents of the public key
    remote.run(f"echo {public_key} >> ~/.ssh/authorized_keys")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 129.114.26.122: b'bc02df4a10b1e53395b677dbc744efab'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 129.114.27.104: b'c81e5deb6e04f04bc70ee256ff87fa4a'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 129.114.25.34: b'f4854d6c5ddea191fdb5317bad52d2e8'
  warnings.warn(


#### Install keys for SSH access between all servers

In [13]:
for node_ip in nodes.values():
    remote = chi.ssh.Remote(node_ip)
    remote.run('ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -q -N ""')
    public_key = remote.run('cat ~/.ssh/id_rsa.pub').tail("stdout")[2:]
    public_key = public_key.replace("\n", "")
    for other_node_ip in nodes.values():
        remote = chi.ssh.Remote(other_node_ip)
        remote.run(f'echo {public_key} >> ~/.ssh/authorized_keys') 

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDHesV07lAeZ3HvYKf5tnTqYYFQEqWX+ghKgxeCr9ghJD+BGyO5bM9fZXrfH2XX9MlobSiporQ5nAk/3QM+0YRD5czMJrBRViZBb0d6gs02y/wlnA7/B7u4QaGtFMfbcgbu2pMlt1tvOGEFQQ1tKNz2GyQAyGZRJp43uHbfaYf7IHDXqD1K225KJLx6+MhAVZywxTIoRyqSmgIxpXQIn7T/gLPv3gmzQRyku5uMZ98kStO9YusSo74ZWR9hx4JJ0ObQW6QtC59NinfzzN1yobhH8DQAEdqfCwFo3OApDyLgJP4BtERzOqD7/bc3x88kIJVqW58uzHvTs4GWOs13p2gDB8ri0hjY9KxDFU8I+ByfFFbfaXu+JPPEHDsj+SadZULMaUc2GASloEpgQRbygTHaRgzg5lSdjeQ1zFrp+xYL7+P0neQPynPVMlfv5N3c8QhryUo6mY9M/bxiqfz0IYB5kRXytBf/qqeFIqe+z76G/uhGrE6WAPAKC1s9ieXP1BNkoWyeTdZq9nRAzkTJt17juIbVx8kBROQ3xVidxttTr4sjOdju+oXgl0iVgOhwLmASg/jPZikSWVlUi3cmQdsJxTiLbNANHRGUaTZxuQAifk5ouvwFYOGh2XtiobSRyW/cLTjLSPOcfaiRdmFMR07jd8SjuIKfgePodRTiKcgK3w== cc@node-0-cp3793-nyu-edu-k8s-ml
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCsOwVBH8sWVjzJRdtyocYkNSQDgFQR+wDHA24ZUN9Brh3ZNVroN8gzl0O4kK+23m+kHrY9h4W1WustVhO5uB6vZ+BgLM3EF4QSkf1I2AKrxt7YZkNj4VPGtLyq/wQ5J9H1BYBOwhaBrwIOYLhWlv02a8hwMXsn3a+kiYXp6Sd2z0VyMkh2kiZ4GPwRctg2gnvnt87j3mYmxVw

#### Store the IP address of each host

In [14]:
physical_ips=[]
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    physical_ips.append(remote.run('hostname -I').tail("stdout")[2:-1])

192.168.1.20 
192.168.1.21 
192.168.1.22 


#### Use Kubespray to prepare a Kubernetes cluster

In [15]:
ansible_node = nodes[0]
remote = chi.ssh.Remote(ansible_node)
remote.run("sudo apt install virtualenv")
remote.run("virtualenv -p python3 myenv")
remote.run("git clone https://github.com/kubernetes-sigs/kubespray.git")
remote.run("source myenv/bin/activate; cd kubespray; pip3 install -r requirements.txt")
remote.run("cd kubespray; cp -rfp inventory/sample inventory/mycluster")
remote.run("rm -rf kubespray/contrib/inventory_builder/inventory.py")
remote.run("rm -rf kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run("rm -rf kubespray/inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml")
remote.run("wget https://raw.githubusercontent.com/indianspeedster/kubespray/master/inventory/sample/group_vars/k8s_cluster/k8s-cluster.yml -O kubespray/inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py -O kubespray/contrib/inventory_builder/inventory.py")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml -O kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run(f"source myenv/bin/activate; declare -a IPS=({physical_ips[0]} {physical_ips[1]} {physical_ips[2]});"+"cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}")

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  virtualenv
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 2144 B of archives.
After this operation, 23.6 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/universe amd64 virtualenv all 20.0.17-1ubuntu0.4 [2144 B]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 2144 B in 0s (15.1 kB/s)
Selecting previously unselected package virtualenv.
(Reading database ... 82514 files and directories currently installed.)
Preparing to unpack .../virtualenv_20.0.17-1ubuntu0.4_all.deb ...
Unpacking virtualenv (20.0.17-1ubuntu0.4) ...
Setting up virtualenv (20.0.17-1ubuntu0.4) ...
created virtual environment CPython3.8.10.final.0-64 in 1681ms
  creator CPython3Posix(dest=/home/cc/myenv, clear=False, global=False)
  seeder FromAppData(download=False, pip=latest, setuptools=latest, wheel=latest, pkg_resources=latest, via=copy, app_data_dir=/home/cc/.local/share/virtualenv/seed-app-data/v1.0.1.debian.1)
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator


Cloning into 'kubespray'...


  Created wheel for ansible: filename=ansible-5.7.1-py3-none-any.whl size=61777681 sha256=4129d6cb0decbed67ea4460473971bfb51a767db73e13b12ebfb60be11be050c
  Stored in directory: /home/cc/.cache/pip/wheels/02/07/2a/7b3eb5d79e268b769b0910cded0d524b4647ae5bc19f3ebb70
  Created wheel for ansible-core: filename=ansible_core-2.12.10-py3-none-any.whl size=2079008 sha256=237ef2aff39f9aa00a4fb2ee37eeaf32b7ad02987012ba00cf23bcfa9c05a774
  Stored in directory: /home/cc/.cache/pip/wheels/eb/61/b4/118d4eac8be0248f90ddde117bee70e10d3a37be86786289a5
Successfully built ansible ansible-core


--2023-06-03 20:07:38--  https://raw.githubusercontent.com/indianspeedster/kubespray/master/inventory/sample/group_vars/k8s_cluster/k8s-cluster.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15449 (15K) [text/plain]
Saving to: ‘kubespray/inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml’

     0K .......... .....                                      100% 34.6M=0s

2023-06-03 20:07:38 (34.6 MB/s) - ‘kubespray/inventory/mycluster/group_vars/k8s_cluster/k8s-cluster.yml’ saved [15449/15449]

--2023-06-03 20:07:38--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubuse

DEBUG: Adding group all
DEBUG: Adding group kube_control_plane
DEBUG: Adding group kube_node
DEBUG: Adding group etcd
DEBUG: Adding group k8s_cluster
DEBUG: Adding group calico_rr
DEBUG: adding host node-0 to group all
DEBUG: adding host node-1 to group all
DEBUG: adding host node-2 to group all
DEBUG: adding host node-0 to group etcd
DEBUG: adding host node-1 to group etcd
DEBUG: adding host node-2 to group etcd
DEBUG: adding host node-0 to group kube_control_plane
DEBUG: adding host node-1 to group kube_control_plane
DEBUG: adding host node-0 to group kube_node
DEBUG: adding host node-1 to group kube_node
DEBUG: adding host node-2 to group kube_node


<Result cmd='source myenv/bin/activate; declare -a IPS=(192.168.1.20 192.168.1.21 192.168.1.22);cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}' exited=0>

In [16]:
remote.run("source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml")

[WARNING]: Skipping callback plugin 'ara_default', unable to load



PLAY [localhost] ***************************************************************
Saturday 03 June 2023  20:08:17 +0000 (0:00:00.043)       0:00:00.043 ********* 

TASK [Check 2.12.0 <= Ansible version < 2.13.0] ********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Saturday 03 June 2023  20:08:17 +0000 (0:00:00.060)       0:00:00.103 ********* 

TASK [Check that python netaddr is installed] **********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Saturday 03 June 2023  20:08:18 +0000 (0:00:00.138)       0:00:00.241 ********* 

TASK [Check that jinja is not too old (install via pip)] ***********************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}


[WARNING]: Could not match supplied host pattern, ignoring: kube-master



PLAY [Add kube-master nodes to kube_control_plane] *****************************
skipping: no hosts matched


[WARNING]: Could not match supplied host pattern, ignoring: kube-node


...

[WARNING]: could not parse environment value, skipping: ['{{ proxy_disable_env
}}', "{{ proxy_env if container_manager == 'containerd' else omit }}"]
[WARNING]: could not parse environment value, skipping: ['{{ proxy_disable_env
}}', "{{ proxy_env if container_manager == 'containerd' else omit }}"]
[WARNING]: could not parse environment value, skipping: ['{{ proxy_disable_env
}}', "{{ proxy_env if container_manager == 'containerd' else omit }}"]



TASK [download : download_container | Download image if required] **************
changed: [node-0]
changed: [node-2]
changed: [node-1]
Saturday 03 June 2023  20:15:25 +0000 (0:00:09.970)       0:07:07.440 ********* 
Saturday 03 June 2023  20:15:25 +0000 (0:00:00.205)       0:07:07.645 ********* 
Saturday 03 June 2023  20:15:25 +0000 (0:00:00.097)       0:07:07.742 ********* 
Saturday 03 June 2023  20:15:25 +0000 (0:00:00.102)       0:07:07.845 ********* 
Saturday 03 June 2023  20:15:25 +0000 (0:00:00.100)       0:07:07.945 ********* 

TASK [download : download_container | Remove container image from cache] *******
ok: [node-0]
ok: [node-1]
ok: [node-2]

PLAY [etcd:kube_control_plane] *************************************************
Saturday 03 June 2023  20:15:26 +0000 (0:00:00.630)       0:07:08.575 ********* 
Saturday 03 June 2023  20:15:26 +0000 (0:00:00.114)       0:07:08.689 ********* 
Saturday 03 June 2023  20:15:26 +0000 (0:00:00.054)       0:07:08.744 ********* 
Saturday 03 J

[WARNING]: Skipped '/etc/ssl/etcd/ssl' path due to this access issue:
'/etc/ssl/etcd/ssl' is not a directory



TASK [etcd : Check_certs | Register certs that have already been generated on first etcd node] ***
ok: [node-0]
Saturday 03 June 2023  20:15:31 +0000 (0:00:00.265)       0:07:14.031 ********* 

TASK [etcd : Check_certs | Set default value for 'sync_certs', 'gen_certs' and 'etcd_secret_changed' to false] ***
ok: [node-0]
ok: [node-1]
ok: [node-2]
Saturday 03 June 2023  20:15:31 +0000 (0:00:00.125)       0:07:14.157 ********* 

TASK [etcd : Check certs | Register ca and etcd admin/member certs on etcd hosts] ***
ok: [node-0] => (item=ca.pem)
ok: [node-1] => (item=ca.pem)
ok: [node-2] => (item=ca.pem)
ok: [node-0] => (item=member-node-0.pem)
ok: [node-1] => (item=member-node-1.pem)
ok: [node-2] => (item=member-node-2.pem)
ok: [node-0] => (item=member-node-0-key.pem)
ok: [node-1] => (item=member-node-1-key.pem)
ok: [node-2] => (item=member-node-2-key.pem)
ok: [node-0] => (item=admin-node-0.pem)
ok: [node-1] => (item=admin-node-1.pem)
ok: [node-2] => (item=admin-node-2.pem)
ok: [node-0] =>

<Result cmd='source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml' exited=0>

#### Provide relevent access to kubectl and Docker

In [17]:
import chi.ssh
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube")

<Result cmd='sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube' exited=0>

In [18]:
import chi.ssh
for node_ip in nodes.values():
    remote = chi.ssh.Remote(node_ip)
    remote.run("sudo ufw allow 32000")

Rule added
Rule added (v6)
Rule added
Rule added (v6)
Rule added
Rule added (v6)


In [19]:
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo groupadd docker; sudo usermod -aG docker $USER")
remote = chi.ssh.Remote(nodes[0])

groupadd: group 'docker' already exists


#### Start local Docker registry

In [20]:
remote.run("docker run -d -p 5000:5000 --restart always --name registry registry:2")
for node_ip in nodes.values():
    remote = chi.ssh.Remote(node_ip)
    remote.run("sudo wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json -O /etc/docker/daemon.json")
    remote.run("sudo service docker restart")

Unable to find image 'registry:2' locally
2: Pulling from library/registry
8a49fdb3b6a5: Pulling fs layer
58116d8bf569: Pulling fs layer
4cb4a93be51c: Pulling fs layer
cbdeff65a266: Pulling fs layer
6b102b34ed3d: Pulling fs layer
cbdeff65a266: Waiting
6b102b34ed3d: Waiting
58116d8bf569: Download complete
8a49fdb3b6a5: Download complete
4cb4a93be51c: Verifying Checksum
4cb4a93be51c: Download complete
cbdeff65a266: Verifying Checksum
cbdeff65a266: Download complete
6b102b34ed3d: Verifying Checksum
6b102b34ed3d: Download complete
8a49fdb3b6a5: Pull complete
58116d8bf569: Pull complete
4cb4a93be51c: Pull complete
cbdeff65a266: Pull complete
6b102b34ed3d: Pull complete
Digest: sha256:20d084723c951e377e1a2a5b3df316173a845e300d57ccdd8ae3ab2da3439746
Status: Downloaded newer image for registry:2


2063584c0e0e07324afb46e97a4c9e465b72adbacc49e5707c416d8952402a06


--2023-06-03 20:31:49--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65 [text/plain]
Saving to: ‘/etc/docker/daemon.json’

     0K                                                       100% 2.14M=0s

2023-06-03 20:31:49 (2.14 MB/s) - ‘/etc/docker/daemon.json’ saved [65/65]

--2023-06-03 20:32:00--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65 [text/pla

#### Check installation 

In [21]:
remote = chi.ssh.Remote(nodes[0])
remote.run("kubectl get nodes")

NAME     STATUS   ROLES           AGE   VERSION
node-0   Ready    control-plane   14m   v1.26.5
node-1   Ready    control-plane   14m   v1.26.5
node-2   Ready    <none>          13m   v1.26.5


<Result cmd='kubectl get nodes' exited=0>

In [22]:
remote.run("docker run hello-world")

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
719385e32844: Pulling fs layer
719385e32844: Download complete
719385e32844: Pull complete
Digest: sha256:fc6cf906cbfa013e80938cdf0bb199fbdbb86d6e3e013783e5a766f50f5dbce0
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

# Delete resources

### Delete server

In [ ]:
if True: 
    for i in servers:
      chi.server.delete_server(i.id)

### Delete network and routers

In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass
